# Lesson 3: Handling AI Interactions with the Chat Service Layer


In the previous lesson, we explored the `ChatManager` class, which plays a crucial role in managing chat data within our application. Now, we will take the next step in our journey by building the **Chat Service Layer**. This layer is essential for integrating the language model with chat sessions, allowing us to process user messages and generate AI responses. By the end of this lesson, you will understand how to set up the `ChatService` class, create chat sessions, and process messages using OpenAI’s API.

The service layer acts as a bridge between the model layer (where data is managed) and the AI model (which generates responses). It is responsible for orchestrating the flow of data and ensuring that user interactions are handled smoothly. Let’s dive into the details of setting up this important component.

---

## Setting Up the `ChatService` Class

The `ChatService` class is the heart of our service layer. It manages chat sessions and interacts with the OpenAI client to generate AI responses.

### Initialization

```python
import uuid
from openai import OpenAI
from models.chat import ChatManager

class ChatService:
    def __init__(self):
        self.chat_manager   = ChatManager()
        self.openai_client  = OpenAI()
        self.system_prompt  = self.load_system_prompt('data/system_prompt.txt')
```

- We instantiate `ChatManager` to handle chat data.
- We initialize the OpenAI client.
- We load the `system_prompt` from a file (using a helper method).

---

## Loading the System Prompt

The **system prompt** guides the AI’s behavior by providing context and instructions.

```python
def load_system_prompt(self, file_path: str) -> str:
    """Load the system prompt from file."""
    try:
        with open(file_path, 'r') as f:
            return f.read()
    except Exception as e:
        print(f"Error loading system prompt: {e}")
        return "You are a helpful assistant."
```

- Reads the prompt from `file_path`.
- Falls back to a default prompt on error, ensuring continuity.

---

## Creating a New Chat Session

To start a conversation, we need a unique chat session.

```python
def create_chat(self, user_id: str) -> str:
    """Create a new chat session."""
    chat_id = str(uuid.uuid4())
    self.chat_manager.create_chat(user_id, chat_id, self.system_prompt)
    return chat_id
```

- Generates a `chat_id` using `uuid`.
- Calls `ChatManager.create_chat(...)` with `user_id`, `chat_id`, and `system_prompt`.

---

## Processing User Messages

The `process_message` method handles incoming user messages, requests an AI completion, and updates the chat history.

### Steps

1. **Retrieve the chat**  
   Use `get_chat`; throw an error if not found.  
2. **Add the user message**  
   Append to the chat history.  
3. **Generate AI response**  
   Send the full conversation (system prompt + messages) to OpenAI.  
4. **Store and return the AI response**  
   Append the assistant’s reply to the history.  
5. **Error handling**  
   Wrap the OpenAI call in `try/except` to surface errors gracefully.

### Implementation

```python
def process_message(self, user_id: str, chat_id: str, message: str) -> str:
    """Process a user message and get AI response."""
    
    # 1. Retrieve the chat
    chat = self.chat_manager.get_chat(user_id, chat_id)
    if not chat:
        raise ValueError("Chat not found")
    
    # 2. Add user message to history
    self.chat_manager.add_message(user_id, chat_id, "user", message)
    
    try:
        # 3. Get AI response
        conversation = self.chat_manager.get_conversation(user_id, chat_id)
        response = self.openai_client.chat.completions.create(
            model="gpt-4",
            messages=conversation,
            temperature=0.7,
            max_tokens=500
        )
        ai_message = response.choices[0].message.content
        
        # 4. Add AI response to history
        self.chat_manager.add_message(user_id, chat_id, "assistant", ai_message)
        return ai_message

    except Exception as e:
        # 5. Handle errors
        raise RuntimeError(f"Error getting AI response: {e}")
```

> **Note:** We set `temperature=0.7` for balanced creativity and `max_tokens=500` to allow detailed yet concise replies—ideal for a customer‑service context.

---

## Example: Simulating a Chat Session

Below is how you’d use `ChatService` in `main.py` to create a session and exchange a message.

```python
from services.chat_service import ChatService

# Initialize the chat service
chat_service = ChatService()

# Simulate a user ID and create session
user_id = "user123"
chat_id  = chat_service.create_chat(user_id)
print(f"Chat session created with ID: {chat_id}")

# Simulate sending a messageHere’s the full implementation of `ChatService` with `create_chat`, plus a quick test:

```python
# services/chat_service.py

import uuid
from openai import OpenAI
from models.chat import ChatManager

class ChatService:
    def __init__(self):
        # Initialize ChatManager to handle chat data
        self.chat_manager   = ChatManager()
        # Initialize OpenAI client (you can configure with api_key/env as needed)
        self.openai_client  = OpenAI()
        # Load the system prompt for all new chats
        self.system_prompt  = self.load_system_prompt('data/system_prompt.txt')

    def load_system_prompt(self, file_path: str) -> str:
        """Load the system prompt from file."""
        try:
            with open(file_path, 'r') as f:
                return f.read()
        except Exception as e:
            print(f"Error loading system prompt: {e}")
            return "You are a helpful assistant."

    def create_chat(self, user_id: str) -> str:
        """
        Create a new chat session for a given user.
        Returns the generated chat_id.
        """
        # 1. Generate a unique chat ID
        chat_id = str(uuid.uuid4())
        # 2. Use ChatManager to create and store the new chat
        self.chat_manager.create_chat(user_id, chat_id, self.system_prompt)
        # 3. Return the new chat_id
        return chat_id
```

And here’s how you’d test it in a separate script (e.g. `main.py`):

```python
# main.py

from services.chat_service import ChatService

# 1) Initialize the chat service
chat_service = ChatService()

# 2) Define a sample user ID
user_id = "user123"

# 3) Create a new chat session and capture the chat_id
chat_id = chat_service.create_chat(user_id)

# 4) Print out the chat ID to verify
print(f"Chat session created with ID: {chat_id}")
```

**When you run `main.py`, you should see output similar to:**

```
Chat session created with ID: e3b0c442-98fc-4a5f-b13c-6d7f9b2a1234
```

This confirms that `create_chat` is generating a UUID, storing the session via `ChatManager`, and returning the new `chat_id`.
    print(f"AI Response: {ai_response}")
except Exception as e:
    print(f"Error: {e}")
```

**Sample output:**

```text
Chat session created with ID: 01a17870-8a4f-4b6f-a3ce-f04e1136d597
AI Response: Hello! I'm here to help with any questions or concerns you might have regarding our IT services. How can I assist you today?
```

---

## Summary and Next Steps

In this lesson, you learned how to:

- Set up the `ChatService` class and initialize its components  
- Load and fall back on a system prompt  
- Create unique chat sessions via `uuid`  
- Process user messages end‑to‑end with OpenAI’s API  
- Handle errors gracefully  

**Practice exercise:** Experiment by adding different `temperature` and `max_tokens` values, or extend `ChatService` with logging and metrics. Happy coding!  


## Implementing Chat Session Creation in ChatService

Let's begin by implementing the create_chat method within the ChatService class. This method is essential for generating a unique chat ID and initializing a chat session. Here's what you need to do:

Implement the create_chat method within the ChatService class.

Use the uuid module to generate a unique chat ID.
Utilize the ChatManager to create a chat session, associating it with the user ID, chat ID, and system prompt.
Ensure the method returns the generated chat ID.
Test the chat creation using the service.

Initialize an instance of ChatService.
Define a sample user ID to simulate a user.
Call the create_chat method with the user ID and store the returned chat ID.
Print the chat ID to verify the successful creation of the chat session.
This task will guide you through the process of creating and managing chat sessions, a fundamental aspect of building a chatbot service.


```python
import uuid
from openai import OpenAI
from models.chat import ChatManager


class ChatService:
    def __init__(self):
        # TODO: Initialize ChatManager to handle chat data
        self.chat_manager = 
        self.system_prompt = self.load_system_prompt('data/system_prompt.txt')

    def load_system_prompt(self, file_path: str) -> str:
        """Load the system prompt from file."""
        try:
            with open(file_path, 'r') as f:
                return f.read()
        except Exception as e:
            print(f"Error loading system prompt: {e}")
            return "You are a helpful assistant."

    # TODO: Define the create_chat method
    # - Parameters: user_id
    # - Generate a unique chat ID using uuid
    # - Use chat_manager to create a chat session with user_id, chat_id, and system_prompt
    # - Return the chat_id


```

```python
from services.chat_service import ChatService

# TODO: Initialize a ChatService instance

# TODO: Define a variable with a sample user identifier, e.g., "user123"

# TODO: Call the create_chat method with the user ID and store the chat ID

# TODO: Print the chat ID


```

Here’s the full implementation of `ChatService` with `create_chat`, plus a quick test:

```python
# services/chat_service.py

import uuid
from openai import OpenAI
from models.chat import ChatManager

class ChatService:
    def __init__(self):
        # Initialize ChatManager to handle chat data
        self.chat_manager   = ChatManager()
        # Initialize OpenAI client (you can configure with api_key/env as needed)
        self.openai_client  = OpenAI()
        # Load the system prompt for all new chats
        self.system_prompt  = self.load_system_prompt('data/system_prompt.txt')

    def load_system_prompt(self, file_path: str) -> str:
        """Load the system prompt from file."""
        try:
            with open(file_path, 'r') as f:
                return f.read()
        except Exception as e:
            print(f"Error loading system prompt: {e}")
            return "You are a helpful assistant."

    def create_chat(self, user_id: str) -> str:
        """
        Create a new chat session for a given user.
        Returns the generated chat_id.
        """
        # 1. Generate a unique chat ID
        chat_id = str(uuid.uuid4())
        # 2. Use ChatManager to create and store the new chat
        self.chat_manager.create_chat(user_id, chat_id, self.system_prompt)
        # 3. Return the new chat_id
        return chat_id
```

And here’s how you’d test it in a separate script (e.g. `main.py`):

```python
# main.py

from services.chat_service import ChatService

# 1) Initialize the chat service
chat_service = ChatService()

# 2) Define a sample user ID
user_id = "user123"

# 3) Create a new chat session and capture the chat_id
chat_id = chat_service.create_chat(user_id)

# 4) Print out the chat ID to verify
print(f"Chat session created with ID: {chat_id}")
```

**When you run `main.py`, you should see output similar to:**

```
Chat session created with ID: e3b0c442-98fc-4a5f-b13c-6d7f9b2a1234
```

This confirms that `create_chat` is generating a UUID, storing the session via `ChatManager`, and returning the new `chat_id`.

## Implementing Message Processing in Chat Service

Cosmo • Yesterday
Read message out loud
You've done well setting up the ChatService class and creating chat sessions. Now, let's begin implementing the process_message method to handle user messages. For now, we will focus on storing the user's message without getting a response from the AI. Here's what you need to do:

Implement the process_message method within the ChatService class.

Check if the chat session exists using the get_chat method from ChatManager.
If the chat doesn't exist, raise a ValueError with the message "Chat not found".
Add the user's message to the chat history using the add_message method from ChatManager.
Return a string indicating the message was processed, e.g., "Message processed".
Test the process_message method by simulating a user message, ensuring the chat history is updated correctly, and handle any exceptions by printing them if the chat session does not exist.

This task will ensure that your chat service can effectively handle user messages, setting the stage for integrating AI responses in the future.

```python
import uuid
from openai import OpenAI
from models.chat import ChatManager


class ChatService:
    def __init__(self):
        self.chat_manager = ChatManager()
        self.system_prompt = self.load_system_prompt('data/system_prompt.txt')

    def load_system_prompt(self, file_path: str) -> str:
        """Load the system prompt from file."""
        try:
            with open(file_path, 'r') as f:
                return f.read()
        except Exception as e:
            print(f"Error loading system prompt: {e}")
            return "You are a helpful assistant."

    def create_chat(self, user_id: str) -> str:
        """Create a new chat session."""
        chat_id = str(uuid.uuid4())
        self.chat_manager.create_chat(user_id, chat_id, self.system_prompt)
        return chat_id

    # TODO: Define the process_message method
    # - Parameters: user_id, chat_id, message
    # - Check if the chat session exists using get_chat
    #   - If the chat doesn't exist, raise a ValueError with "Chat not found"
    # - Add the user's message to the chat history using add_message
    # - Return a string indicating the message was processed


```

```python
from services.chat_service import ChatService

# Initialize the chat service
chat_service = ChatService()

# Simulate a user ID
user_id = "user123"

# Create a new chat session
chat_id = chat_service.create_chat(user_id)

# TODO: Simulate sending a message and handle potential exceptions


```

I'll help you implement the `process_message` method in the ChatService class and test it. Here's how to complete the code:

First, let's implement the `process_message` method in the ChatService class:

```python
def process_message(self, user_id: str, chat_id: str, message: str) -> str:
    """Process a user message for a specific chat session."""
    # Check if the chat session exists
    chat = self.chat_manager.get_chat(user_id, chat_id)
    if not chat:
        raise ValueError("Chat not found")

    # Add the user's message to the chat history
    self.chat_manager.add_message(user_id, chat_id, "user", message)

    # Return a confirmation message
    return "Message processed"
```

Now, let's complete the test code to simulate sending a message and handle exceptions:

```python
from services.chat_service import ChatService

# Initialize the chat service
chat_service = ChatService()

# Simulate a user ID
user_id = "user123"

# Create a new chat session
chat_id = chat_service.create_chat(user_id)
print(f"Created chat session with ID: {chat_id}")

# Simulate sending a message and handle potential exceptions
try:
    # Send a valid message
    message = "Hello, how can you help me today?"
    result = chat_service.process_message(user_id, chat_id, message)
    print(f"Result: {result}")

    # Verify the message was added to chat history
    chat = chat_service.chat_manager.get_chat(user_id, chat_id)
    print(f"Chat history: {chat['messages']}")

    # Test with an invalid chat ID
    invalid_chat_id = "invalid_id"
    result = chat_service.process_message(user_id, invalid_chat_id, "This should fail")
except ValueError as e:
    print(f"Error: {e}")
```

This implementation:
1. Checks if the chat exists using the chat manager
2. Raises a ValueError if the chat is not found
3. Adds the user's message to the chat history
4. Returns a confirmation message
5. Includes test code that verifies successful message processing and proper error handling

## Enhancing AI Response Generation in Chat Service

Nice work on setting up the process_message method! Now, let's enhance it to generate AI responses. Your task is to add the missing code that interacts with the OpenAI client to obtain the AI response and update the chat history.

Here's what you need to do in the ChatService class:

Instantiate the OpenAI client to interact with the AI model.
Retrieve the conversation using get_conversation from ChatManager.
Use the OpenAI client to generate a response from "gpt-4" based on the conversation, set the temperature to 0.7 and the maximum tokens to 500 to balance creativity and coherence in responses.
Extract the AI's response from the OpenAI client response.
Add the AI's response to the chat history.
Return the AI's response.
Remember to handle exceptions during the OpenAI client interaction by raising a RuntimeError with an appropriate message.

The main code is pre-configured, allowing you to test the integration seamlessly. Keep going; you're doing well!

```python
import uuid
from openai import OpenAI
from models.chat import ChatManager


class ChatService:
    def __init__(self):
        self.chat_manager = ChatManager()
        # TODO: Initialize the OpenAI client to interact with the AI model
        self.openai_client = 
        self.system_prompt = self.load_system_prompt('data/system_prompt.txt')

    def load_system_prompt(self, file_path: str) -> str:
        """Load the system prompt from file."""
        try:
            with open(file_path, 'r') as f:
                return f.read()
        except Exception as e:
            print(f"Error loading system prompt: {e}")
            return "You are a helpful assistant."

    def create_chat(self, user_id: str) -> str:
        """Create a new chat session."""
        chat_id = str(uuid.uuid4())
        self.chat_manager.create_chat(user_id, chat_id, self.system_prompt)
        return chat_id

    def process_message(self, user_id: str, chat_id: str, message: str) -> str:
        """Process a user message and get AI response."""
        chat = self.chat_manager.get_chat(user_id, chat_id)
        if not chat:
            raise ValueError("Chat not found")

        # Add user message
        self.chat_manager.add_message(user_id, chat_id, "user", message)

        try:
            # TODO: Retrieve the conversation using the ChatManager

            # TODO: Use the OpenAI client to generate a response based on the conversation
            # - Set the model to "gpt-4"
            # - Pass the conversation as the messages parameter to provide context
            # - Use a temperature of 0.7 to balance creativity and coherence in responses
            # - Limit the response length to 500 tokens

            # TODO: Extract the AI's response from the OpenAI client response

            # TODO: Add the AI's response to the chat history

            # TODO: Return the AI's response

        except Exception as e:
            # TODO: Handle exceptions and raise a RuntimeError with an appropriate message


```


```python
from services.chat_service import ChatService

# Initialize the chat service
chat_service = ChatService()

# Simulate a user ID
user_id = "user123"

# Create a new chat session
chat_id = chat_service.create_chat(user_id)
print(f"Chat session created with ID: {chat_id}")

# Simulate sending a message
user_message = "How can I contact your company by email?"
print(f"User Message: {user_message}")

try:
    ai_response = chat_service.process_message(user_id, chat_id, user_message)
    print(f"AI Response: {ai_response}")
except Exception as e:
    print(f"Error: {e}")



```

Here's the enhanced implementation of the ChatService class with AI response generation:


```python
import uuid
from openai import OpenAI
from models.chat import ChatManager


class ChatService:
    def __init__(self):
        self.chat_manager = ChatManager()
        # Initialize the OpenAI client to interact with the AI model
        self.openai_client = OpenAI()
        self.system_prompt = self.load_system_prompt('data/system_prompt.txt')

    def load_system_prompt(self, file_path: str) -> str:
        """Load the system prompt from file."""
        try:
            with open(file_path, 'r') as f:
                return f.read()
        except Exception as e:
            print(f"Error loading system prompt: {e}")
            return "You are a helpful assistant."

    def create_chat(self, user_id: str) -> str:
        """Create a new chat session."""
        chat_id = str(uuid.uuid4())
        self.chat_manager.create_chat(user_id, chat_id, self.system_prompt)
        return chat_id

    def process_message(self, user_id: str, chat_id: str, message: str) -> str:
        """Process a user message and get AI response."""
        chat = self.chat_manager.get_chat(user_id, chat_id)
        if not chat:
            raise ValueError("Chat not found")

        # Add user message
        self.chat_manager.add_message(user_id, chat_id, "user", message)

        try:
            # Retrieve the conversation using the ChatManager
            conversation = self.chat_manager.get_conversation(user_id, chat_id)

            # Use the OpenAI client to generate a response based on the conversation
            response = self.openai_client.chat.completions.create(
                model="gpt-4",
                messages=conversation,
                temperature=0.7,
                max_tokens=500
            )

            # Extract the AI's response from the OpenAI client response
            ai_response = response.choices[0].message.content

            # Add the AI's response to the chat history
            self.chat_manager.add_message(user_id, chat_id, "assistant", ai_response)

            # Return the AI's response
            return ai_response

        except Exception as e:
            # Handle exceptions and raise a RuntimeError with an appropriate message
            raise RuntimeError(f"Failed to generate AI response: {str(e)}")
```

This implementation:

1. Initializes the OpenAI client in the constructor
2. Retrieves the conversation history from the ChatManager
3. Makes a request to the OpenAI API with the specified parameters:
   - Uses the "gpt-4" model
   - Passes the full conversation history
   - Sets temperature to 0.7 for balanced creativity
   - Limits responses to 500 tokens
4. Extracts the AI's response from the API response
5. Adds the AI's response to the chat history
6. Returns the AI's response to the caller
7. Handles exceptions by raising a RuntimeError with details about what went wrong

The implementation follows best practices for error handling and maintains the conversation flow between the user and the AI assistant.

## Enhancing AI Context Retention in Multi-Turn Conversations

You've successfully set up the chat service and processed the initial message. Now, let's enhance the AI's capability by testing its ability to retain context across multiple interactions. This will ensure the AI can handle multi-turn conversations effectively.

Here's what you need to do:

After sending the initial message, define a follow-up question that requires the AI to recall the context of the previous message. For example, you might ask: "Is there another way?" Print this follow-up question to the console.

Use the process_message method to send this follow-up question to the AI.

Print the AI's response to the follow-up question to verify its ability to maintain context across multiple messages.

This task will help you verify the AI's ability to maintain context across multiple messages. Keep going; you're doing well!


```python
from services.chat_service import ChatService

# Initialize the chat service
chat_service = ChatService()

# Simulate a user ID
user_id = "user123"

# Create a new chat session
chat_id = chat_service.create_chat(user_id)
print(f"Chat session created with ID: {chat_id}")

# Define a user message
user_message = "How can I contact your company by email?"
print(f"User Message: {user_message}")

try:
    ai_response = chat_service.process_message(user_id, chat_id, user_message)
    print(f"AI Response: {ai_response}")
except Exception as e:
    print(f"Error: {e}")

# TODO: Define and print a follow-up question
# TODO: Send the follow-up question to test context retention
# TODO: Print the AI's response to the follow-up question

```


```python
import uuid
from openai import OpenAI
from models.chat import ChatManager


class ChatService:
    def __init__(self):
        self.chat_manager = ChatManager()
        self.openai_client = OpenAI()
        self.system_prompt = self.load_system_prompt('data/system_prompt.txt')

    def load_system_prompt(self, file_path: str) -> str:
        """Load the system prompt from file."""
        try:
            with open(file_path, 'r') as f:
                return f.read()
        except Exception as e:
            print(f"Error loading system prompt: {e}")
            return "You are a helpful assistant."

    def create_chat(self, user_id: str) -> str:
        """Create a new chat session."""
        chat_id = str(uuid.uuid4())
        self.chat_manager.create_chat(user_id, chat_id, self.system_prompt)
        return chat_id

    def process_message(self, user_id: str, chat_id: str, message: str) -> str:
        """Process a user message and get AI response."""
        chat = self.chat_manager.get_chat(user_id, chat_id)
        if not chat:
            raise ValueError("Chat not found")

        # Add user message
        self.chat_manager.add_message(user_id, chat_id, "user", message)

        try:
            # Get AI response
            conversation = self.chat_manager.get_conversation(user_id, chat_id)

            response = self.openai_client.chat.completions.create(
                model="gpt-4",
                messages=conversation,
                temperature=0.7,
                max_tokens=500
            )

            ai_message = response.choices[0].message.content

            # Add AI response to chat history
            self.chat_manager.add_message(user_id, chat_id, "assistant", ai_message)

            return ai_message

        except Exception as e:
            raise RuntimeError(f"Error getting AI response: {str(e)}")



```

Here's the enhanced code to test the AI's context retention in multi-turn conversations:
```python
from services.chat_service import ChatService

# Initialize the chat service
chat_service = ChatService()

# Simulate a user ID
user_id = "user123"

# Create a new chat session
chat_id = chat_service.create_chat(user_id)
print(f"Chat session created with ID: {chat_id}")

# Define a user message
user_message = "How can I contact your company by email?"
print(f"User Message: {user_message}")

try:
    ai_response = chat_service.process_message(user_id, chat_id, user_message)
    print(f"AI Response: {ai_response}")

    # Define and print a follow-up question that requires context retention
    follow_up_question = "Is there another way to contact you besides email?"
    print(f"\nFollow-up Question: {follow_up_question}")

    # Send the follow-up question to test context retention
    follow_up_response = chat_service.process_message(user_id, chat_id, follow_up_question)

    # Print the AI's response to the follow-up question
    print(f"AI Response to Follow-up: {follow_up_response}")

except Exception as e:
    print(f"Error: {e}")
```

This implementation:

1. Creates a chat session and sends the initial message about contacting the company by email
2. Defines a follow-up question that requires the AI to remember the context of the previous conversation (asking about alternative contact methods)
3. Sends the follow-up question using the same chat session ID to maintain conversation history
4. Prints the AI's response to verify it maintains context across multiple interactions

The follow-up question specifically tests if the AI remembers that the conversation is about contacting the company, without explicitly mentioning "the company" again. This demonstrates the AI's ability to maintain context across multiple turns in the conversation.

## Managing Multiple Chat Sessions for a User

You've made great progress in setting up the ChatService class and processing user messages. Now, let's focus on creating multiple chat sessions for the same user to ensure that each session has a cleared context.

Here's what you need to do:

Create a Second Chat Session: Use the create_chat method to create a new chat session for the same user. This will ensure that the user can have multiple independent conversations.

Process a Message in the Second Chat: Simulate sending a message in the newly created chat session using the process_message method. This will test that the chat session is functioning correctly and independently from the first session.

This task will guide you through the process of creating and managing chat sessions, a fundamental aspect of building a chatbot service. Keep up the momentum!


```python
from services.chat_service import ChatService

# Initialize the chat service
chat_service = ChatService()

# Simulate a user ID
user_id = "user123"

# Create a new chat session
chat_id = chat_service.create_chat(user_id)
print(f"Chat session created with ID: {chat_id}")

# Simulate sending a message
user_message = "Hello, how are you?"

try:
    ai_response = chat_service.process_message(user_id, chat_id, user_message)
    print(f"AI Response: {ai_response}")
except Exception as e:
    print(f"Error: {e}")

# TODO: Create a second chat session for the same user and process a new message



```


```python
import uuid
from openai import OpenAI
from models.chat import ChatManager


class ChatService:
    def __init__(self):
        self.chat_manager = ChatManager()
        self.openai_client = OpenAI()
        self.system_prompt = self.load_system_prompt('data/system_prompt.txt')

    def load_system_prompt(self, file_path: str) -> str:
        """Load the system prompt from file."""
        try:
            with open(file_path, 'r') as f:
                return f.read()
        except Exception as e:
            print(f"Error loading system prompt: {e}")
            return "You are a helpful assistant."

    def create_chat(self, user_id: str) -> str:
        """Create a new chat session."""
        chat_id = str(uuid.uuid4())
        self.chat_manager.create_chat(user_id, chat_id, self.system_prompt)
        return chat_id

    def process_message(self, user_id: str, chat_id: str, message: str) -> str:
        """Process a user message and get AI response."""
        chat = self.chat_manager.get_chat(user_id, chat_id)
        if not chat:
            raise ValueError("Chat not found")

        # Add user message
        self.chat_manager.add_message(user_id, chat_id, "user", message)

        try:
            # Get AI response
            conversation = self.chat_manager.get_conversation(user_id, chat_id)

            response = self.openai_client.chat.completions.create(
                model="gpt-4",
                messages=conversation,
                temperature=0.7,
                max_tokens=500
            )

            ai_message = response.choices[0].message.content

            # Add AI response to chat history
            self.chat_manager.add_message(user_id, chat_id, "assistant", ai_message)

            return ai_message

        except Exception as e:
            raise RuntimeError(f"Error getting AI response: {str(e)}")


```

Here's the implementation for managing multiple chat sessions for a user:

from services.chat_service import ChatService

```python
# Initialize the chat service
chat_service = ChatService()

# Simulate a user ID
user_id = "user123"

# Create a new chat session
chat_id = chat_service.create_chat(user_id)
print(f"First chat session created with ID: {chat_id}")

# Simulate sending a message
user_message = "Hello, how are you?"
print(f"First chat - User Message: {user_message}")

try:
    ai_response = chat_service.process_message(user_id, chat_id, user_message)
    print(f"First chat - AI Response: {ai_response}")
except Exception as e:
    print(f"Error: {e}")

# Create a second chat session for the same user
second_chat_id = chat_service.create_chat(user_id)
print(f"\nSecond chat session created with ID: {second_chat_id}")

# Process a message in the second chat
second_message = "What's the weather like today?"
print(f"Second chat - User Message: {second_message}")

try:
    second_ai_response = chat_service.process_message(user_id, second_chat_id, second_message)
    print(f"Second chat - AI Response: {second_ai_response}")

    # Verify that the contexts are separate by sending a follow-up in the first chat
    follow_up_message = "Can you continue our previous conversation?"
    print(f"\nFirst chat - Follow-up Message: {follow_up_message}")

    follow_up_response = chat_service.process_message(user_id, chat_id, follow_up_message)
    print(f"First chat - AI Response: {follow_up_response}")
except Exception as e:
    print(f"Error: {e}")
```

This implementation:

1. Creates an initial chat session and processes a message
2. Creates a second chat session for the same user with a different chat ID
3. Processes a completely different message in the second chat session
4. Adds a follow-up message to the first chat to verify that the contexts remain separate
5. Prints all interactions to clearly show the separation between chat sessions

The code demonstrates how a single user can maintain multiple independent conversations with the AI, each with its own context and history. This is essential for applications where users might want to have separate conversations about different topics without context bleeding between them.